In [ ]:
from transformers import BertTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset
import numpy as np
from sklearn.metrics import accuracy_score
import torch
import torch.nn.utils.prune as prune
from pyswarm import pso

# Load IMDb dataset
dataset = load_dataset('imdb')

# Use a small subset of the dataset for faster experimentation
small_train_dataset = dataset['train'].shuffle(seed=42).select(range(1000))
small_test_dataset = dataset['test'].shuffle(seed=42).select(range(1000))

# Load TinyBERT model and tokenizer using AutoModelForSequenceClassification
tokenizer = BertTokenizer.from_pretrained('prajjwal1/bert-tiny')
model = AutoModelForSequenceClassification.from_pretrained('prajjwal1/bert-tiny', num_labels=2)

# Make all model parameters contiguous
for param in model.parameters():
    param.data = param.data.contiguous()

# Tokenize data with fixed sequence length
def tokenize_function(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True, max_length=128)  # Define max_length

tokenized_train = small_train_dataset.map(tokenize_function, batched=True)
tokenized_test = small_test_dataset.map(tokenize_function, batched=True)

# Remove columns that are not tensors
tokenized_train = tokenized_train.remove_columns(["text"])
tokenized_test = tokenized_test.remove_columns(["text"])

# Define compute metrics function
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = accuracy_score(labels, predictions)
    return {"accuracy": accuracy}

# Define PSO fitness function (objective function)
def fitness_function(hyperparams):
    lr, batch_size, epochs = hyperparams
    batch_size = int(batch_size)
    epochs = int(epochs)

    # Reduce training arguments for faster execution
    training_args = TrainingArguments(
        output_dir='./results',
        num_train_epochs=epochs,
        per_device_train_batch_size=batch_size,
        learning_rate=lr,
        evaluation_strategy="steps",
        eval_steps=50,  # Evaluate every 50 steps to save time
        save_steps=50,  # Save model checkpoint every 50 steps
        logging_steps=50,
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_train,
        eval_dataset=tokenized_test,
        compute_metrics=compute_metrics  # Pass the compute metrics function
    )

    # Make sure model parameters are contiguous before training
    for param in model.parameters():
        param.data = param.data.contiguous()

    # Train and evaluate the model
    trainer.train()
    eval_results = trainer.evaluate()

    # Return negative accuracy to minimize the fitness function
    return -eval_results['eval_accuracy']

# PSO to optimize hyperparameters: learning rate, batch size, epochs
lb = [1e-5, 8, 1]  # Lower bounds for hyperparameters
ub = [1e-3, 16, 2]  # Upper bounds for hyperparameters, reduced search space

best_hyperparams, _ = pso(fitness_function, lb, ub, swarmsize=5, maxiter=3)

# Output optimized hyperparameters
print("Best hyperparameters:", best_hyperparams)

# Train final model with optimized hyperparameters
final_lr, final_batch_size, final_epochs = best_hyperparams
final_batch_size = int(final_batch_size)
final_epochs = int(final_epochs)

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=final_epochs,
    per_device_train_batch_size=final_batch_size,
    learning_rate=final_lr,
    evaluation_strategy="epoch"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    compute_metrics=compute_metrics,
     # Pass the compute metrics function
)

# Make all parameters contiguous before final training
for param in model.parameters():
    param.data = param.data.contiguous()

# Apply L1 unstructured pruning to all linear layers in the model
def apply_l1_pruning(model, amount=0.4):
    for name, module in model.named_modules():
        if isinstance(module, torch.nn.Linear):
            prune.l1_unstructured(module, name='weight', amount=amount)
            prune.remove(module, 'weight')  # Remove the pruning mask, making it permanent

# Apply L1 pruning with 40% sparsity
apply_l1_pruning(model, amount=0.4)

# Train the final pruned model and evaluate
trainer.train()
final_results = trainer.evaluate()
print("Final accuracy after pruning:", final_results['eval_accuracy'])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

...

Step,Training Loss,Validation Loss,Accuracy
50,0.014300,2.738878,0.741000
100,0.461900,2.303587,0.743000


Stopping search: maximum iterations reached --> 3
Best hyperparameters: [1.20512667e-04 1.43749001e+01 1.87742404e+00]


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,2.037895,0.747000


Final accuracy after pruning: 0.747


In [ ]:
# Define the directory where the model and tokenizer will be saved
output_dir = "./saved_model"

# Save the model and tokenizer after training
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

# Optionally, you can also save the training arguments and optimizer states if needed
trainer.save_model(output_dir)  # This will save the model, tokenizer, and additional components


In [ ]:
from transformers import BertTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset
import numpy as np
from sklearn.metrics import accuracy_score
import torch
import torch.nn.utils.prune as prune
from pyswarm import pso

# Load IMDb dataset
dataset = load_dataset('imdb')

# Use a small subset of the dataset for faster experimentation
small_train_dataset = dataset['train'].shuffle(seed=42).select(range(1000))
small_test_dataset = dataset['test'].shuffle(seed=42).select(range(1000))

# Load TinyBERT model and tokenizer using AutoModelForSequenceClassification
tokenizer2 = BertTokenizer.from_pretrained('prajjwal1/bert-tiny')
model2 = AutoModelForSequenceClassification.from_pretrained('prajjwal1/bert-tiny', num_labels=2)

# Make all model parameters contiguous
for param in model2.parameters():
    param.data = param.data.contiguous()

# Tokenize data with fixed sequence length
def tokenize_function(examples):
    return tokenizer2(examples['text'], padding="max_length", truncation=True, max_length=128)  # Define max_length

tokenized_train = small_train_dataset.map(tokenize_function, batched=True)
tokenized_test = small_test_dataset.map(tokenize_function, batched=True)

# Remove columns that are not tensors
tokenized_train = tokenized_train.remove_columns(["text"])
tokenized_test = tokenized_test.remove_columns(["text"])

# Define compute metrics function
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = accuracy_score(labels, predictions)
    return {"accuracy": accuracy}

# Define PSO fitness function (objective function)
def fitness_function(hyperparams):
    lr, batch_size, epochs = hyperparams
    batch_size = int(batch_size)
    epochs = int(epochs)

    # Reduce training arguments for faster execution
    training_args = TrainingArguments(
        output_dir='./results',
        num_train_epochs=epochs,
        per_device_train_batch_size=batch_size,
        learning_rate=lr,
        evaluation_strategy="steps",
        eval_steps=50,  # Evaluate every 50 steps to save time
        save_steps=50,  # Save model checkpoint every 50 steps
        logging_steps=50,
    )

    trainer = Trainer(
        model=model2,
        args=training_args,
        train_dataset=tokenized_train,
        eval_dataset=tokenized_test,
        compute_metrics=compute_metrics  # Pass the compute metrics function
    )

    # Make sure model parameters are contiguous before training
    for param in model2.parameters():
        param.data = param.data.contiguous()

    # Train and evaluate the model
    trainer.train()
    eval_results = trainer.evaluate()

    # Return negative accuracy to minimize the fitness function
    return -eval_results['eval_accuracy']

# PSO to optimize hyperparameters: learning rate, batch size, epochs
lb = [1e-6, 8, 2] # Lower bounds for hyperparameters
ub = [1e-2, 32, 4]  # Upper bounds for hyperparameters, reduced search space

best_hyperparams, _ = pso(fitness_function, lb, ub, swarmsize=10, maxiter=10)

# Output optimized hyperparameters
print("Best hyperparameters:", best_hyperparams)

# Train final model with optimized hyperparameters
final_lr, final_batch_size, final_epochs = best_hyperparams
final_batch_size = int(final_batch_size)
final_epochs = int(final_epochs)

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=final_epochs,
    per_device_train_batch_size=final_batch_size,
    learning_rate=final_lr,
    evaluation_strategy="epoch"
)

trainer = Trainer(
    model=model2,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    compute_metrics=compute_metrics,
     # Pass the compute metrics function
)

# Make all parameters contiguous before final training
for param in model2.parameters():
    param.data = param.data.contiguous()

# Apply L1 unstructured pruning to all linear layers in the model
def apply_l1_pruning(model, amount=0.4):
    for name, module in model.named_modules():
        if isinstance(module, torch.nn.Linear):
            prune.l1_unstructured(module, name='weight', amount=amount)
            prune.remove(module, 'weight')  # Remove the pruning mask, making it permanent

# Apply L1 pruning with 40% sparsity
apply_l1_pruning(model2, amount=0.4)

# Train the final pruned model and evaluate
trainer.train()
final_results = trainer.evaluate()
print("Final accuracy after pruning:", final_results['eval_accuracy'])

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-tiny and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Step,Training Loss,Validation Loss,Accuracy
50,0.720000,0.692737,0.502000
100,0.693000,0.684405,0.575000


...

Step,Training Loss,Validation Loss,Accuracy
50,0.345600,1.115117,0.518000
100,0.317500,0.984769,0.525000


Stopping search: maximum iterations reached --> 10
Best hyperparameters: [3.74602665e-03 3.08171434e+01 3.46398788e+00]


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.975413,0.524000
2,No log,0.923616,0.527000
3,No log,0.986001,0.519000


Final accuracy after pruning: 0.519


In [ ]:
from transformers import BertTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset
import numpy as np
from sklearn.metrics import accuracy_score
import torch
import torch.nn.utils.prune as prune
from pyswarm import pso

# Load IMDb dataset
dataset = load_dataset('imdb')

# Use a small subset of the dataset for faster experimentation
small_train_dataset = dataset['train'].shuffle(seed=42).select(range(1000))
small_test_dataset = dataset['test'].shuffle(seed=42).select(range(1000))

# Load TinyBERT model and tokenizer using AutoModelForSequenceClassification
tokenizer = BertTokenizer.from_pretrained('prajjwal1/bert-tiny')
model = AutoModelForSequenceClassification.from_pretrained('prajjwal1/bert-tiny', num_labels=2)

# Make all model parameters contiguous
for param in model.parameters():
    param.data = param.data.contiguous()

# Tokenize data with fixed sequence length
def tokenize_function(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True, max_length=128)  # Define max_length

tokenized_train = small_train_dataset.map(tokenize_function, batched=True)
tokenized_test = small_test_dataset.map(tokenize_function, batched=True)

# Remove columns that are not tensors
tokenized_train = tokenized_train.remove_columns(["text"])
tokenized_test = tokenized_test.remove_columns(["text"])

# Define compute metrics function
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = accuracy_score(labels, predictions)
    return {"accuracy": accuracy}

# Define PSO fitness function (objective function)
def fitness_function(hyperparams):
    lr, batch_size, epochs = hyperparams
    batch_size = int(batch_size)
    epochs = int(epochs)

    # Reduce training arguments for faster execution
    training_args = TrainingArguments(
        output_dir='./results',
        num_train_epochs=epochs,
        per_device_train_batch_size=batch_size,
        learning_rate=lr,
        evaluation_strategy="steps",
        eval_steps=50,  # Evaluate every 50 steps to save time
        save_steps=50,  # Save model checkpoint every 50 steps
        logging_steps=50,
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_train,
        eval_dataset=tokenized_test,
        compute_metrics=compute_metrics  # Pass the compute metrics function
    )

    # Make sure model parameters are contiguous before training
    for param in model.parameters():
        param.data = param.data.contiguous()

    # Train and evaluate the model
    trainer.train()
    eval_results = trainer.evaluate()

    # Return negative accuracy to minimize the fitness function
    return -eval_results['eval_accuracy']

# PSO to optimize hyperparameters: learning rate, batch size, epochs
lb = [1e-5, 8, 1]  # Lower bounds for hyperparameters
ub = [1e-3, 16, 2]  # Upper bounds for hyperparameters, reduced search space

best_hyperparams, _ = pso(fitness_function, lb, ub, swarmsize=5, maxiter=3)

# Output optimized hyperparameters
print("Best hyperparameters:", best_hyperparams)

# Train final model with optimized hyperparameters
final_lr, final_batch_size, final_epochs = best_hyperparams
final_batch_size = int(final_batch_size)
final_epochs = int(final_epochs)

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=final_epochs,
    per_device_train_batch_size=final_batch_size,
    learning_rate=final_lr,
    evaluation_strategy="epoch",
    lr_scheduler_type="cosine",  # Use cosine decay scheduler
    warmup_ratio=0.1  # Warm up for the first 10% of steps
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    compute_metrics=compute_metrics,
     # Pass the compute metrics function
)

# Make all parameters contiguous before final training
for param in model.parameters():
    param.data = param.data.contiguous()

# Apply L1 unstructured pruning to all linear layers in the model
def apply_l1_pruning(model, amount=0.4):
    for name, module in model.named_modules():
        if isinstance(module, torch.nn.Linear):
            prune.l1_unstructured(module, name='weight', amount=amount)
            prune.remove(module, 'weight')  # Remove the pruning mask, making it permanent

# Apply L1 pruning with 40% sparsity
apply_l1_pruning(model, amount=0.4)

# Train the final pruned model and evaluate
trainer.train()
final_results = trainer.evaluate()
print("Final accuracy after pruning:", final_results['eval_accuracy'])

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-tiny and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Step,Training Loss,Validation Loss,Accuracy
50,0.692000,0.676135,0.603000


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Step,Training Loss,Validation Loss,Accuracy
50,0.654400,0.740582,0.550000
100,0.617700,0.626256,0.670000


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


...

Step,Training Loss,Validation Loss,Accuracy
50,0.000000,4.992571,0.745000


Stopping search: maximum iterations reached --> 3
Best hyperparameters: [3.93913342e-04 1.49004917e+01 1.86924486e+00]


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,3.992364,0.740000


Final accuracy after pruning: 0.74
